# IMPORTS

In [1]:
# !pip install transformers datasets scikit-learn pandas
# !pip install accelerate

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import torch
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

=======================================================================================================

# DATA

In [3]:
# для проверки пайплайна
dataset = load_dataset("imdb")
train_df = pd.DataFrame(dataset['train']).sample(10000, random_state=42).reset_index(drop=True)
test_df = pd.DataFrame(dataset['test']).sample(2000, random_state=42).reset_index(drop=True)

In [4]:
len(train_df)

10000

In [5]:
train_df

,text,label
0,"Dumb is as dumb does, in this thoroughly unint...",0
1,I dug out from my garage some old musicals and...,1
2,After watching this movie I was honestly disap...,0
3,This movie was nominated for best picture but ...,1
4,Just like Al Gore shook us up with his painful...,1
...,...,...
9995,"OK, here is my personal list of top Nicktoons ...",1
9996,The surprise nominee of this year's Best Anima...,1
9997,"As others have said, ""No, Luciano"" is a more a...",0
9998,"Tierney's an authentic tough guy, but this mov...",0


In [6]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_length=512, text_col="text", label_col="label"):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.text_col = text_col
        self.label_col = label_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, self.text_col]
        label = self.df.loc[idx, self.label_col]

        tokens = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
  
        item = {key: val.squeeze() for key, val in tokens.items()}
        item[self.label_col] = torch.tensor(label, dtype=torch.long)
        return item

In [7]:
def prepare_dataset(df, tokenizer, text_column="text", label_column="label", max_length=512):
    
    dataset = Dataset.from_pandas(df[[text_column, label_column]])
    
    def tokenize_function(example):
        return tokenizer(
            example[text_column], 
            padding="max_length", 
            truncation=True, 
            max_length=max_length
        )
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset = tokenized_dataset.remove_columns([text_column])
    tokenized_dataset.set_format("torch")

    return tokenized_dataset

In [8]:
def generate_classification_report(model, tokenizer, df, text_column="text", label_column="label", max_length=512, batch_size=32):
    predictions = []
    labels = df[label_column].tolist()
    
    for i in tqdm(range(0, len(df), batch_size)):
        batch_texts = df[text_column].iloc[i:i + batch_size].tolist()
        
        tokens = tokenizer(
            batch_texts,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        
        tokens = {k: v.to(model.device) for k, v in tokens.items()}

        with torch.no_grad():
            outputs = model(**tokens)
            batch_predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            predictions.extend(batch_predictions)
        del tokens
        torch.cuda.empty_cache()
    
    report = classification_report(labels, predictions, target_names=["Class 0", "Class 1"])
    
    return report

===============================================================================================

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average="weighted")
    # accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    
    return {"f1": f1,
            "precision": precision,
            "recall": recall,
           }

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [11]:
model_name = "cointegrated/LaBSE-en-ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/LaBSE-en-ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
print(generate_classification_report(model, tokenizer, test_df, text_column="text", label_column="label"))

  0%|          | 0/63 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


              precision    recall  f1-score   support

     Class 0       0.54      0.65      0.59      1040
     Class 1       0.51      0.39      0.44       960

    accuracy                           0.53      2000
   macro avg       0.52      0.52      0.52      2000
weighted avg       0.52      0.53      0.52      2000



In [11]:
train_dataset = TextDataset(train_df, tokenizer, max_length=512)
test_dataset = TextDataset(test_df, tokenizer, max_length=512)
# train_dataset = prepare_dataset(train_df, tokenizer, text_column="text", label_column="label", max_length=512)
# test_dataset = prepare_dataset(test_df, tokenizer, text_column="text", label_column="label", max_length=512)

In [12]:
training_args = TrainingArguments(
    output_dir='./results',                      # Директория для сохранения результатов
    eval_strategy="epoch",                 # Оценка модели на каждой эпохе
    learning_rate=2e-5,                          # Скорость обучения
    per_device_train_batch_size=32,              # Размер батча для обучения
    per_device_eval_batch_size=32,               # Размер батча для валидации
    num_train_epochs=10,                         # Количество эпох
    weight_decay=0.00001,                           # Коэффициент регуляризации
    logging_dir='./logs',                        # Директория для логов (TensorBoard)
    logging_steps=50,                           # Логирование каждые 500 шагов
    load_best_model_at_end=True,                 # Загрузка лучшей модели по окончанию обучения
    save_total_limit=2,                          # Сохраняем только 2 лучшие модели
    metric_for_best_model="f1",                  # Ключевая метрика для выбора лучшей модели
    greater_is_better=True,                      # Лучшая модель — та, где метрика больше
    save_strategy="epoch",                       # Сохраняем модель на каждой эпохе
    report_to="tensorboard",                     # Используем TensorBoard для логирования
    optim="adamw_torch",                         # Явно указываем AdamW как оптимизатор
    warmup_steps=100,
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,                 # Тренировочный датасет
    eval_dataset=test_dataset,                    # Валидационный датасет
    tokenizer=tokenizer,                         # Токенизатор
    compute_metrics=compute_metrics,             # Функция вычисления метрик
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Ранняя остановка (2 эпохи)
    # data_collator=data_collator
)

# Запуск тренировки
trainer.train()

C:\Temp\ipykernel_9156\100174452.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\ProgramData\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 